#  버스 대기 시간 03/04

##  

 Jake 의 [blog](http://jakevdp.github.io/blog/2018/09/13/waiting-time-paradox/)
 
  ## 문제: 탑승자가 정류장에 도착해서  버스를 기다리는 시간의 평균값
    
1. Model :  
    
    - 탑승자가 정류장 도착하는 시간 
    - 버스가 정류장 도착 시간   
    
2. 1 차 가정 : 단순 생각 
    
    - 탑승자 정류장 도착 시간:  uniform distribution인 random variable 
    - 버스의 정류장 도착 시간의  간격은  상수:   예)  10분    
    - Waiting Time = (10/2)분
      
    
3. 2차 가정 (실제상황을 묘사)

 3.1 가정
   
    - 버스의 정류장  도착 시간은 주변 상황에 따라 변함 : 버스 도착 시간 간격은 r.v. 
    - 이 경우 두 개의 random variables 에 대한 문제이다    
        
 3.2 개념(Inspection Paradox) 
    
    버스가 도착하는 시간이 random 하면  버스 도착 시간 간격이 변하게 된다. 이 경우 탑승자가 도착해서 
    해서 기다리는 시간은 시간 간격이 긴 경우 기다리는 시간은 더 길게 되고. 따라서 탑승자가 긴 시간 동안 
    도착하는 확률도 커지게 된다. 즉 탑승자의 도착 시간은 uniform 하게 도착하지만,기다리는 시간이 길면
    더 많은 사람들이 기다리고, 따라서 더 길게 된다.  (예;  observing / observed, 평균 교실 학급 수
    인원[Allen Downey.](http://allendowney.blogspot.com/2015/08/the-inspection-paradox-is-everywhere.html)
    )
    
 3.3 Simulation 
    simulation 결과 = 탑승자 대기 시간 = 10분 !! 
    
 3.4 검증
    
    simulation 결과를 확인해 봄. 
    

    
    

## 버스 도착 시간 = bus_arrival_times,     
    -rand.rand(1000000): 1000000개 (0,1) uniform r.v.    
    -버스 도착 시간의 간격 = 10 분
    -rand.rand(N) : (0,1) 사이의 값을 갖는 N개 r.v. 
    -np.sort(rand.rand(N)) : 순서가 정해진 N 개 r.v.
    -N * tau * np.sort(rand.rand(N)) :  인접한 두 개의 r.v.'s 의  간격  평균이 tau인 N개 r.v.
        
    -평균 탑승자 대기 시간 = 10분 

# Jake's [blog](http://jakevdp.github.io/blog/2018/09/13/waiting-time-paradox/)

```python
import numpy as np

N = 1000000  # number of buses
tau = 10  # average minutes between arrivals

rand = np.random.RandomState(42)  # universal random seed
bus_arrival_times = N * tau * np.sort(rand.rand(N))

intervals = np.diff(bus_arrival_times)
intervals.mean()

# waiting time 계산 
def simulate_wait_times(arrival_times,
                        rseed=8675309,  # Jenny's random seed
                        n_passengers=1000000):
    rand = np.random.RandomState(rseed)
    
    arrival_times = np.asarray(arrival_times)
    passenger_times = arrival_times.max() * rand.rand(n_passengers)

    # find the index of the next bus for each simulated passenger
    i = np.searchsorted(arrival_times, passenger_times, side='right')

    return arrival_times[i] - passenger_times


wait_times = simulate_wait_times(bus_arrival_times)
wait_times.mean()
```

 3.4 검증
    
 - $T$ = 버스 도착시간 간격, 평균이 10분인 r.v.
 - what is    __$p(T)$__ ?    

$$
E[T] = \int_0^\infty T~p(T)~dT = 10분(위 simulation 결과 )
$$
 - 탑승자 대기 시간  확률 = $P_탑$ 
    - $P_탑$는  $p(T)$ 에 비례하고 또 __$T$__ 도 비례한다
    
    - $P_탑$ $\propto$ $Tp(T)$ 
    
    - $$
        p_{탑}(T) = \frac{T~p(T)}{\int_0^\infty T~p(T)~dT}
      $$
      
    - 탑승자 대기 시간의 평균 = $E(W) = E(T)/2 = \frac{1}{2}\int_0^\infty T~p_{exp}(T)~dT$

$$
E[W] = \frac{E[T^2]}{2E[T]}
$$

 ##  따라서 :  What is  $p(T)$ and  How to compute the integrals.

 3.5 $p(T)$  simulation으로 구하기 

     
### Jake's [blog](http://jakevdp.github.io/blog/2018/09/13/waiting-time-paradox/)

```python
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn')

plt.hist(intervals, bins=np.arange(80), density=True)
plt.axvline(intervals.mean(), color='black', linestyle='dotted')
plt.xlabel('Interval between arrivals (minutes)')
plt.ylabel('Probability density');

```

###  markdown image upload  - image file이 local directory 에 있어야 된다

![prob](untitled.png)

그림에서 버스 도착 시간 간격의 확률 밀도(histogram으로 작성)는 exponential 함수이다. exponential 을 갖는 확률 밀도 함수는  *Poisson Process*가 유사하다. 또 는 Beta didtribution도 있다. 만약 Poisson이라고 가정하면 double  check로 일정 시간동안 도착한 버스노 Poisson Process를 가지므로 확인한다

버스 도착시간은 평균적으로 10분에 한 대 씩이므로 한 시간에 몇 대가 도착했는 지 확인한다

```python
from scipy.stats import poisson

# count the number of arrivals in 1-hour bins
binsize = 60
binned_arrivals = np.bincount((bus_arrival_times // binsize).astype(int))
```

시간당 최대 도착한 버스수는 
```python
max =np.bincount((bus_arrival_times//60).astype(int)).max()
max  
```

19대 이므로 histogram의 최대 x 축의 크기를 20으로 하고 simualtion 한 histogram과 poisson process를 비교하면  

```python
x = np.arange(20)

# plot the results
plt.hist(binned_arrivals, bins=x - 0.5, density=True, alpha=0.5, label='simulation')
plt.plot(x, poisson(binsize / tau).pmf(x), 'ok', label='Poisson prediction')
plt.xlabel('Number of arrivals per hour')
plt.ylabel('frequency')
plt.legend();
```

![prob](poisson_pmf.png)

poisson distribution과 유사하다. 따라서 $p(T)$는 
$$
p(T) = \frac{1}{\tau}e^{-T/\tau}
$$
이 것을 이용하여 탑승자가 대기하는 평균시간을 계산하면 
$$
E[W] = \frac{\int_0^\infty T^2~e^{-T/\tau}}{2\int_0^\infty T~e^{-T/\tau}} = \frac{2\tau^3}{2(\tau^2)} = \tau
$$

즉 이 경우 탑승자의 평균 대기 시간은 버스가 도착하는 평균 시간 간격과 일치한다!!
이 결과는 버스가 도착하는 시간 간격이 일정할 경우 5분 이었으나 , 시간 간격이 평균 10분인 Poisson Process로 모델링한한
경우는 10이다. 어느 것이 실제 상황과 유사한가?
Poisson Process는 Memoryless process로 이 전 시간에 얼마나 길 던지 상관 없이  탑승자의 대기 시간은 항상 10분이다.
즉 탑승자는 이 전에 얼마 만큼 기다렸는 지에 상관이 없다. 지금 시점에서 대기 시간이 10분이다.


### 실제 data를 가지고 분석
4.실제 data


Seattle의 실제 버스 정류장에서 버스 schedule data [Seatlle_arrival_times.csv](arrival_times.csv)를 살펴 보자
실제 버스가 도착한 시간 간격은 Poisson 보다는 Gaussina에 가깝고 또 탑승자 대기 시간의 평균은 버스 도착 시간 
간격보다 그 것의 반 값 더하기 1~ 2분 정도이다.


5. # 교훈: 가정은 검정 해야 한다. 

![prob](real.png)